In [ ]:
import torch
from models.feature_extractor import TradingFeatureExtractor


# --- Пример использования ---
if __name__ == "__main__":
    # Параметры
    B, T, F_in = 4, 256, 5
    F_out = 256
    
    # Создание модели
    model = TradingFeatureExtractor(input_size=F_in, feature_size=F_out)
    print(model)
    
    # Примерные входные данные
    x = torch.randn(B, T, F_in)
    
    # Прогон через автоэнкодер
    reconstructed_x = model(x)
    
    print(f"Входные данные: {x.shape}") # [4, 256, 5]
    print(f"Реконструированные данные: {reconstructed_x.shape}") # [4, 256, 5]
    
    # Только извлечение признаков
    features_only = model.extract_features(x)
    print(f"Извлеченные признаки (только): {features_only.shape}") # [4, 256, 128]


### Обучение модели

In [1]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from safetensors.torch import save_model
from tqdm import tqdm

from models.feature_extractor import TradingFeatureExtractor
from utils.dataset import TradingDataset
from utils.normalizer import TradingNormalizer

# Для логирования (опционально)
try:
    from torch.utils.tensorboard import SummaryWriter
    TENSORBOARD_AVAILABLE = True
except ImportError:
    print("TensorBoard не найден. Логирование метрик отключено.")
    TENSORBOARD_AVAILABLE = False
    SummaryWriter = None



def collate_fn(batch):
    """
    Функция для сборки батча из TradingDataset.
    Извлекает только исторические данные, так как они используются как вход и таргет.
    """
    histories = [item['history'] for item in batch]
    # targets = [item['target'] for item in batch] # Не нужны для автоэнкодера
    tickers = [item['ticker'] for item in batch]

    # Объединяем истории в один тензор
    batch_histories = torch.cat(histories, dim=0) # [B, 256, 5]

    return {
        'history': batch_histories,
        'ticker': tickers
    }



def train_feature_extractor(
    data_path='data/',
    batch_size=32,
    num_epochs=100,
    learning_rate=1e-4,
    save_steps=1000,
    model_save_path='pretrained-extractor',
    tensorboard_log_dir='runs/feature_extractor',
    device='mps',
):
    """
    Основная функция для обучения TradingFeatureExtractor.
    """
    # Определяем устройство
    if device == "cuda" and torch.cuda.is_available():
        device = torch.device("cuda")
    elif device == "mps" and torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print(f"Используемое устройство: {device}")


    # --- Создание датасета и даталоадера ---
    print("Загрузка обучающего датасета...")
    normalizer = TradingNormalizer()

    train_dataset = TradingDataset(
        data_path=data_path, 
        mode='train',
        history_len=256, 
        target_len=32,
        normalizer=normalizer,
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=0,
    )


    # --- Инициализация модели ---
    print("Инициализация модели TradingFeatureExtractor...")
    model = TradingFeatureExtractor(
        input_size=5, 
        feature_size=256
    )
    model.to(device)
    print(f"Модель загружена на {device}")


    # --- Инициализация оптимизатора и функции потерь ---
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    # MSE loss - стандартный выбор для автоэнкодеров
    criterion = torch.nn.MSELoss() 


    # --- Инициализация TensorBoard (если доступно) ---
    writer = None
    if TENSORBOARD_AVAILABLE and tensorboard_log_dir:
        import time
        timestamp = str(int(time.time()))
        run_name = f"run_{timestamp}"
        full_log_dir = os.path.join(tensorboard_log_dir, run_name)
        writer = SummaryWriter(log_dir=full_log_dir)
        print(f"TensorBoard логгер инициализирован. Логи будут в {full_log_dir}")


    # --- Цикл обучения ---
    print("Начало обучения...")
    model.train()
    global_step = 0

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=True)

        for step, batch in enumerate(progress_bar):
            try:
                # 1. Получаем данные
                x = batch['history'].to(device) # [B, 256, 5]
                # В автоэнкодере таргет - это сам вход
                target_x = x.clone() 

                # 2. Прямой проход
                reconstructed_x = model(x) # [B, 256, 5]

                # 3. Вычисление потерь
                # Потери вычисляются между реконструкцией и оригиналом
                loss = criterion(reconstructed_x, target_x)

                # 4. Обратный проход и оптимизация
                optimizer.zero_grad()
                loss.backward()
                # Опционально: добавьте градиентный клиппинг, если наблюдаются проблемы с exploding gradients
                # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                # 5. Логирование
                epoch_loss += loss.item()
                global_step += 1

                # Обновляем прогресс-бар
                progress_bar.set_postfix({'loss': f'{loss.item():.6f}'})

                # Логируем в TensorBoard
                if writer is not None:
                    writer.add_scalar("Loss/batch", loss.item(), global_step)
                    # Можно также логировать градиенты, веса и т.д.

            except Exception as e:
                print(f"\nОшибка на шаге {epoch+1}.{step+1}: {e}")
                # Пропускаем батч в случае ошибки
                optimizer.zero_grad()
                continue

            if global_step > 0 and global_step % save_steps == 0:
                os.makedirs(model_save_path, exist_ok=True)
                model_weights_path = os.path.join(model_save_path, "trading_feature_extractor.safetensors")
                model_config_path = os.path.join(model_save_path, "config.json") 
                try:
                    # Сохраняем веса модели с помощью safetensors
                    save_model(model, model_weights_path)
                    print(f"Веса модели сохранены в {model_weights_path} (формат safetensors)")
                    
                    # Опционально: сохраняем конфигурацию в отдельный JSON файл
                    import json
                    config = {
                        'input_size': model.input_size,
                        'feature_size': model.feature_size,
                    }
                    with open(model_config_path, 'w') as f:
                        json.dump(config, f, indent=2)
                    print(f"Конфигурация модели сохранена в {model_config_path}")
                    
                except Exception as e:
                    print(f"Ошибка при сохранении модели: {e}")

        # --- Конец эпохи ---
        avg_epoch_loss = epoch_loss / len(train_loader)
        print(f"Средний лосс на эпохе {epoch+1}: {avg_epoch_loss:.6f}")

        # Логируем средний лосс эпохи
        if writer is not None:
            writer.add_scalar("Loss/epoch", avg_epoch_loss, epoch + 1)

    # --- Завершение обучения ---
    if writer is not None:
        writer.close()
        print(f"TensorBoard логирование завершено.")

    # --- Сохранение модели ---
    # ========== ИЗМЕНЕНО ==========
    os.makedirs(model_save_path, exist_ok=True)
    model_weights_path = os.path.join(model_save_path, "trading_feature_extractor.safetensors")
    model_config_path = os.path.join(model_save_path, "config.json") # Опционально: сохраняем конфиг
    
    try:
        # Сохраняем веса модели с помощью safetensors
        save_model(model, model_weights_path)
        print(f"Веса модели сохранены в {model_weights_path} (формат safetensors)")
        
        # Опционально: сохраняем конфигурацию в отдельный JSON файл
        import json
        config = {
            'input_size': model.input_size,
            'feature_size': model.feature_size,
            # Можно добавить информацию о конфигурации энкодера/декодера, если она не фиксирована
            'num_epochs': num_epochs,
            'final_loss': avg_epoch_loss,
        }
        with open(model_config_path, 'w') as f:
            json.dump(config, f, indent=2)
        print(f"Конфигурация модели сохранена в {model_config_path}")
        
    except Exception as e:
        print(f"Ошибка при сохранении модели: {e}")
    # =============================

    print("Обучение TradingFeatureExtractor завершено.")


if __name__ == "__main__":
    train_feature_extractor()

Epoch 3/100:  52%|█████▏    | 8306/15848 [10:36<09:52, 12.72it/s, loss=0.001169]

Веса модели сохранены в pretrained-extractor/trading_feature_extractor.safetensors (формат safetensors)
Конфигурация модели сохранена в pretrained-extractor/config.json


Epoch 3/100:  59%|█████▊    | 9306/15848 [11:52<08:20, 13.08it/s, loss=0.001559]

Веса модели сохранены в pretrained-extractor/trading_feature_extractor.safetensors (формат safetensors)
Конфигурация модели сохранена в pretrained-extractor/config.json


Epoch 3/100:  59%|█████▉    | 9367/15848 [11:56<08:15, 13.07it/s, loss=0.001235]


KeyboardInterrupt: 